## Deep Neural Network for MNIST Classification

##### Goal is to write an algorithm that detects handwritten digits
##### Since there are 10 digits, the classification problem will have 10 classes
##### Our goal will be to build a neural network with 2 hidden layers

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Numpy library has functions used for working with arrays and matrices
# faster, clearer and better quality code using Numpy

# Tensorflow is an open-source Machine Learning library, particular focus on training and inference of deep neural networks

# TensorFlow Datasets is a collection of datasets ready to use with TensorFlow

### Loading and Pre-Processing the data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

# 'as_supervised' loads the data in a 2-tuple structure (input and target)
# 'with_info' provides a tuple containing info about the version, features and number of samples of the dataset

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# by default Tensorflow's mnist has training and testing datasets but no validation datasets so we have to split data ourselves

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# creates validation dataset by the number of training samples divided by 10
# tf.cast(x, datatype) converts the variable into a given data type to make sure validation dataset is an intager
# preventing any issues caused by floats

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# stores the number of test samples in a variable

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

# this function scales the data to make the result more numerically stable (e.g. inputs between 0 and 1)
# on images, the matrix is made of intagers between 0 and 255, so we should convert this to floats between 0 and 1

scaled_train_and_validation_data = mnist_train.map(scale)

# this scales the whole train dataset
# dataset.map(function) applies a custom transformation to a given dataset.
# It takes a function which determines the transformation as input

test_data = mnist_test.map(scale)
# also scaling the test dataset

### Shuffle the data

In [3]:
BUFFER_SIZE = 10000
# shuffling should be set to >= number of samples to happen uniformly

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# shuffling the data

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# batch size of 1 = Stochastic Gradient Descent (SGD)
# batch size of # of samples = (single batch) GD
# we want a batch size >1 but < # samples = mini-batch GD
# Increasing batch size will increase completion speed but largely decrease accuracy of model

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
test_data = mnist_test.map(scale)

# this combines the consecutive elements of the dataset into batches

validation_data = validation_data.batch(num_validation_samples)

# the model expects the validation dataset to be in batch-form too so we should override validation data with .batch

test_data = test_data.batch(num_test_samples)

# same with the test data

validation_inputs, validation_targets = next(iter(validation_data))
# the function 'iter' creates an object which can be iterated one element at a time in a loop
# 'next' loads the next batch (next element of an iterable object)

### Model - outline the model

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 200

# there are 784 inputs, 10 output nodes

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
])

# the keras.sequential function is laying down the model used to stack layers
# the keras.layers.Flatten function transforms a tensor into a vector (3D to 2D)
# the keras.layers.Dense function takes inputs and finds the Dot product of inputs and weights and adds the bias
# we also apply the activation function here

### Choose the optimizer and Function Loss

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# categorical_crossentropy expects that you've one-hot encoded the target data already
# sparse_categorical_crossentropy applies the one-hot encoding
# the 3rd argument includes the accuracy we want to measure throughout the training and testing processes

### Training

In [6]:
num_epochs = 5
model.fit(train_data, epochs = num_epochs, validation_data=(validation_inputs, validation_targets), verbose=2)

# provide the model with the set number of epochs (iterations), validation inputs and targets,
# fit the model to the data
# set verbose to 2 to only include important info from each epoch

Epoch 1/5
540/540 - 6s - loss: 0.4985 - accuracy: 0.8570 - val_loss: 0.1568 - val_accuracy: 0.9553
Epoch 2/5
540/540 - 4s - loss: 0.1328 - accuracy: 0.9626 - val_loss: 0.0997 - val_accuracy: 0.9710
Epoch 3/5
540/540 - 4s - loss: 0.0886 - accuracy: 0.9740 - val_loss: 0.0727 - val_accuracy: 0.9783
Epoch 4/5
540/540 - 4s - loss: 0.0697 - accuracy: 0.9793 - val_loss: 0.0800 - val_accuracy: 0.9785
Epoch 5/5
540/540 - 4s - loss: 0.0538 - accuracy: 0.9842 - val_loss: 0.0595 - val_accuracy: 0.9810


In [7]:
# the training accuracy is just the average accuracy across batches whereas validation accuracy is of the whole validation set
# the validation accuracy at the end of the last epoch is the True accuracy of the validation dataset.

# this isn't the full accuracy of the model as we haven't tested the forward propagation of the test dataset
# this is because we may be overfitting the validation dataset by fiddling with the hyperparameters
# the test dataset makes sure our hyperparameters (width, depth, batch size, # epochs etc) don't overfit!,
# as the model hasn't seen this data yet

### Test the Model

In [8]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 732ms/step - loss: 0.1019 - accuracy: 0.9716


In [9]:
# format to make it appear neater and easier to read
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.10. Test accuracy: 97.16%


#### after running the final test for the model, we're no longer allowed to change it as...
#### ... if it is changed it will no longer be a dataset the model has never seen

### Getting a test accuracy very close to the validation accuracy shows that we have not overfit!
### this accuracy is the accuracy expected to get if we test the model in the real world.